## Importamos el ODM y comprobamos que todo se haya inicializado bien

In [98]:
import ODM as odm
import importlib
importlib.reload(odm) #recargamos el modulo por si hemos cambiado algo en el archivo ODM.py

models = odm.initApp()

if models is not None:
    print("Modelo inicializado")
    globals().update(models)

Modelo inicializado


## Insertamos los datos

Para inicializar los datos

In [99]:
#Insertar los siguientes comandos en la Terminal
#mongoimport --db abd --collection Cliente --file Cliente.json --jsonArray
#mongoimport --db abd --collection Producto --file Producto.json --jsonArray
#mongoimport --db abd --collection Compra --file Compra.json --jsonArray
#mongoimport --db abd --collection Proveedor --file Proveedor.json --jsonArray

## 1.	Listado de todas las compras de un cliente

In [100]:
#USAR NICOLAS Y ALICE

def lista_compras_cliente(nombre_cliente = "Nicolas"):

    pipeline = [

        {
            "$match": {
                "cliente" : nombre_cliente
            } 
        },
        {
            "$project": {
                "precio_compra": 1,        
                "direccion_envio": 1,      
            }
        }

    ]

    aggregate_res = Compra.aggregate(pipeline)

    print(f"Compras de {nombre_cliente}")
    for compra in aggregate_res:

        print(f"Precio: {compra['precio_compra']} | Direccion de Envio: {compra['direccion_envio']}")
    print('\n')


nombre_cliente = "Alice"
lista_compras_cliente(nombre_cliente)
lista_compras_cliente()


Compras de Alice
Precio: 1709.46 | Direccion de Envio: {'type': 'Point', 'coordinates': [-73.992285, 40.737871]}
Precio: 1478.47 | Direccion de Envio: {'type': 'Point', 'coordinates': [-73.992285, 40.737871]}
Precio: 1299.97 | Direccion de Envio: {'type': 'Point', 'coordinates': [-73.992285, 40.737871]}


Compras de Nicolas
Precio: 1428.97 | Direccion de Envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]}
Precio: 2249.96 | Direccion de Envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]}
Precio: 1948.97 | Direccion de Envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]}




## 2.	Listado de todos los proveedores para un producto.

In [101]:
def obtener_proveedores_producto(codigo_producto = 12345678):

    pipeline = [

        {
            "$match": {
                "codigo_del_producto" : codigo_producto
            } 
        },
        {
            "$project": {  
                "nombre": 1,
                "proveedores": 1     
            }
        }

    ]

    aggregate_res = Producto.aggregate(pipeline)

    print(f"Proveedores de {codigo_producto}")
    for producto in aggregate_res:

        print(f"Nombre del producto: {producto['nombre']} | Proveedores: {producto['proveedores']}")
    print('\n')

obtener_proveedores_producto()

codigo_producto = 87654321
obtener_proveedores_producto(codigo_producto = codigo_producto)


Proveedores de 12345678
Nombre del producto: Laptop | Proveedores: Digital World


Proveedores de 87654321
Nombre del producto: Smartphone | Proveedores: Gadgets R Us




## 3.	Listado de todos los productos diferentes comprados por un cliente

In [102]:
def lista_productos_comprados(nombre_cliente="Nicolas"):

    pipeline = [

        {
            "$match": {
                "cliente": nombre_cliente
            }
        },
        {
            "$unwind": "$productos"
        },
        {
            "$lookup": {
                "from": "Producto",
                "localField": "productos",
                "foreignField": "codigo_del_producto",
                "as": "producto_info"
            }
        },
        {
            "$unwind": "$producto_info"
        },
        {
            "$group": {
                "_id":"$productos",
                "nombre_producto": { "$first": "$producto_info.nombre" },
                "precio": { "$first": "$producto_info.precio_con_iva" }
                }
        },
        {
            "$project": {
                "codigo_del_producto": "$_id",
                "nombre_producto": "$nombre_producto",
                "precio": "$precio"
            }
        }
    ]

    aggregate_res = Compra.aggregate(pipeline)

    print(f"Productos comprados por {nombre_cliente}")
    for producto in aggregate_res:
        print(f"Codigo del producto: {producto['codigo_del_producto']} | Nombre: {producto['nombre_producto']} | Precio: {producto['precio']}")

    print('\n')

lista_productos_comprados(nombre_cliente = "Alice")
lista_productos_comprados()

Productos comprados por Alice
Codigo del producto: 87654321 | Nombre: Smartphone | Precio: 899.99
Codigo del producto: 78901234 | Nombre: Monitor LED | Precio: 599.99
Codigo del producto: 23456789 | Nombre: Auriculares Bluetooth | Precio: 129.49
Codigo del producto: 12345678 | Nombre: Laptop | Precio: 199.99
Codigo del producto: 67890123 | Nombre: camiseta manga corta | Precio: 79.99
Codigo del producto: 98765432 | Nombre: Televisor 4K | Precio: 499.99
Codigo del producto: 34567890 | Nombre: Reloj inteligente | Precio: 249.0


Productos comprados por Nicolas
Codigo del producto: 67890123 | Nombre: camiseta manga corta | Precio: 79.99
Codigo del producto: 98765432 | Nombre: Televisor 4K | Precio: 499.99
Codigo del producto: 78901234 | Nombre: Monitor LED | Precio: 599.99
Codigo del producto: 12345678 | Nombre: Laptop | Precio: 199.99
Codigo del producto: 87654321 | Nombre: Smartphone | Precio: 899.99
Codigo del producto: 34567890 | Nombre: Reloj inteligente | Precio: 249.0
Codigo del pr

## 4.	Listado de productos vendidos por “Modas Paqui” cuyo nombre contenga “manga corta”.

In [103]:
def lista_productos_paqui():

    pipeline = [

        {
            "$match": {
                "proveedores": "Modas Paqui",
                "nombre": {"$regex": "manga corta"}
            }
        },
        {
            "$project": {
                "codigo_del_producto": 1,
                "nombre": 1,
                "precio_con_iva": 1
            } 
        }
    ]   

    res_aggregate = Producto.aggregate(pipeline)
    
    print("Producto vendidos por Modas Paqui de manga corta")

    for product in res_aggregate:

        print(f"Codigo del producto: {product['codigo_del_producto']} | Nombre: {product['nombre']} | Precio: {product['precio_con_iva']}")

lista_productos_paqui()

Producto vendidos por Modas Paqui de manga corta
Codigo del producto: 67890123 | Nombre: camiseta manga corta | Precio: 79.99


## 5.	Calcular el peso y volumen total de los productos comprados por un cliente un día determinado.

In [104]:
def calcular_volumen(dimensiones):
    # Eliminar "cm" y dividir la cadena en las tres dimensiones
    dim_list = dimensiones.replace(" cm", "").split("x")
    
    # Convertir cada dimensión a un número flotante
    largo, ancho, alto = map(float, dim_list)
    
    # Calcular el volumen
    volumen = largo * ancho * alto
    
    return volumen

def volumen_peso_producto(nombre_cliente = "Nicolas", fecha_compra = "15-10-2024"):

    pipeline = [

        {
            "$match": {
                "cliente": nombre_cliente,
                "fecha_compra": fecha_compra
            }
        },
        {
            "$lookup":{
                "from": "Producto",
                "localField": "productos",
                "foreignField": "codigo_del_producto",
                "as": "producto_info"
            }
        },
        {
            "$unwind": "$producto_info"
        },
        {
            "$project":{
                "nombre": "$producto_info.nombre",
                "peso": "$producto_info.peso",
                "dimensiones": "$producto_info.dimensiones"
            }
        }

    ]

    res_aggregate = Compra.aggregate(pipeline)

    peso_total = 0
    volumen_total = 0

    print(f"Productos comprados por {nombre_cliente} el dia {fecha_compra}")

    for producto in res_aggregate:
        volumen = calcular_volumen(producto['dimensiones'])
        peso_total += producto['peso']
        volumen_total += volumen
        print(f"Nombre del producto: {producto['nombre']} | Peso: {producto['peso']} kg | Volumen: {volumen/1000000} m^3")

    print("\n--- Totales ---")
    print(f"Peso total: {peso_total} kg")
    print(f"Volumen total: {volumen_total/1000000} m^3")



volumen_peso_producto()

Productos comprados por Nicolas el dia 15-10-2024
Nombre del producto: Smartphone | Peso: 0.36 kg | Volumen: 0.38709 m^3
Nombre del producto: Reloj inteligente | Peso: 1.37 kg | Volumen: 0.12648 m^3
Nombre del producto: Tablet | Peso: 0.49 kg | Volumen: 0.267138 m^3
Nombre del producto: camiseta manga corta | Peso: 4.67 kg | Volumen: 0.349272 m^3
Nombre del producto: Smartphone | Peso: 0.36 kg | Volumen: 0.38709 m^3
Nombre del producto: Televisor 4K | Peso: 1.34 kg | Volumen: 0.076912 m^3
Nombre del producto: Reloj inteligente | Peso: 1.37 kg | Volumen: 0.12648 m^3
Nombre del producto: Monitor LED | Peso: 4.46 kg | Volumen: 0.099072 m^3

--- Totales ---
Peso total: 14.420000000000002 kg
Volumen total: 1.819534 m^3


## 6.	Calcular el número medio de envíos por mes y almacén.

In [105]:
def numero_medio_envios_por_mes():
    pipeline = [

        {
            "$lookup": {
                "from": "Producto",
                "localField": "productos",
                "foreignField": "codigo_del_producto",
                "as": "producto_info"
            }
        },
        {
            "$unwind": "$producto_info"
        },
        {
            "$addFields": {
                "mes": {"$month": {"$dateFromString": {"dateString": "$fecha_compra", "format": "%d-%m-%Y"}}},
            }
        },
        {
            "$group": {
                "_id": {
                    "almacen": "$producto_info.almacenes",
                    "mes": "$mes",
                },
                "num_envios": {"$sum": 1}
            }
        },
        {
            "$group": {
                "_id": "$_id.almacen",
                "promedio_envios_por_mes": {"$avg": "$num_envios"}
            }
        }

    ]

    res_aggregate = Compra.aggregate(pipeline)

    print("Promedio de envíos por mes y almacén:\n")
    for almacen in res_aggregate:
        print(f"Almacén: {almacen['_id']} | Promedio de envíos por mes: {almacen['promedio_envios_por_mes']:.2f}")

numero_medio_envios_por_mes()


Promedio de envíos por mes y almacén:

Almacén: Depósito Urbano | Promedio de envíos por mes: 1.00
Almacén: Almacenaje Rápido | Promedio de envíos por mes: 1.00
Almacén: Mercado Central | Promedio de envíos por mes: 1.00
Almacén: Bodega Estrella | Promedio de envíos por mes: 1.71
Almacén: Bodega del Vecino | Promedio de envíos por mes: 1.29
Almacén: Centro de Ofertas | Promedio de envíos por mes: 1.00
Almacén: Surtido Selecto | Promedio de envíos por mes: 1.00
Almacén: Espacio de Compras | Promedio de envíos por mes: 1.00


## 7.	Listado con los tres proveedores con más volumen de facturación. Mostrar proveedor y volumen de facturación.

In [106]:
def top3_proveedores():

    pipeline = [

        {
            "$lookup": {
                "from": "Producto",
                "localField": "productos",
                "foreignField": "codigo_del_producto",
                "as": "producto_info"
            }
        },
        {
            "$unwind": "$producto_info"
        },
        {
            "$group" : {
                "_id": "$producto_info.proveedores",
                "facturacion": {"$sum": "$producto_info.precio_con_iva"}
            }  
        },
        {
            "$sort":{
                "facturacion": -1
            }
        },
        {
            "$limit": 3
        }

    ]

    res_aggregate = Compra.aggregate(pipeline)

    print("Top 3 proveedores:\n")
    for proveedor in res_aggregate:
        print(f"Proveedor: {proveedor["_id"]} | facturacion: {proveedor["facturacion"]}")

top3_proveedores()

Top 3 proveedores:

Proveedor: Gadgets R Us | facturacion: 6893.9400000000005
Proveedor: Modas Paqui | facturacion: 2799.91
Proveedor: Global Tech Supply | facturacion: 2499.95


## 8.	Listado de almacenes cerca de unas coordenadas determinadas (100km de distancia máxima) ordenadas por orden de distancia.

In [107]:
def almacenes_cercanos(longitud=-103.922980, latitud=33.7):
    pipeline = [
        {
            "$geoNear": {
                "near": { 
                    "type": "Point", 
                    "coordinates": [longitud, latitud]
                },
                "distanceField": "direccion_almacenes",
                "maxDistance": 100000,
                "spherical": True
            }
        }
    ]

    res_aggregate = Proveedor.aggregate(pipeline)

    print(f"Distancia almacenes a longitud: {longitud}, latitud: {latitud} (Maximo 100km)\n")
    for proveedor in res_aggregate:
    
        distancia_km = proveedor['direccion_almacenes'] / 1000
        print(f"Almacen: {proveedor['nombre']} | distancia al punto: {distancia_km:.2f} km")

almacenes_cercanos()

Distancia almacenes a longitud: -103.92298, latitud: 33.7 (Maximo 100km)

Almacen: Modas Paqui | distancia al punto: 12.88 km


## 9.	Listado de compras con destino dentro de un polígono cuyos vértices vienen definidos por coordenadas.

In [108]:
def compras_en_poligono(vertices):
    
    poligono = {
        "type": "Polygon",
        "coordinates": [vertices]
    }

    pipeline = [

        {
            "$geoNear": {
                "near": {
                    "type": "Point",
                    "coordinates": [0, 0]  # Este valor no se usa, solo para cumplir la estructura
                },
                "distanceField": "distance",
                "spherical": True,
                "query": {
                    "direccion_envio": {  
                        "$geoWithin": {
                            "$geometry": poligono
                        }
                    }
                }
            }
        }
    ]

    res_aggregate = Compra.aggregate(pipeline)

    print("Compras dentro del polígono:\n")
    for compra in res_aggregate:
        print(f"Cliente: {compra['cliente']} | Direccion de envio: {compra['direccion_envio']} | Precio: {compra['precio_compra']}")

vertices_poligono = [
    [-123.0, 37.5],  
    [-121.5, 37.5],  
    [-121.5, 37.2],  
    [-123.0, 37.2],  
    [-123.0, 37.5]   
]

compras_en_poligono(vertices_poligono)

Compras dentro del polígono:

Cliente: Nicolas | Direccion de envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]} | Precio: 1948.97
Cliente: Nicolas | Direccion de envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]} | Precio: 1428.97
Cliente: Nicolas | Direccion de envio: {'type': 'Point', 'coordinates': [-122.084614, 37.421764]} | Precio: 2249.96


## 10.	Guardar en una tabla nueva el listado de compras que tienen que ser enviados desde un almacén en un día determinado.

In [109]:

def guardar_compras_por_fecha(fecha_envio: str):

    pipeline = [
        {
            "$match": {
                "fecha_compra": fecha_envio  
            }
        },
        {
            "$project": {
                "_id": 0, 
                "cliente": "$nombre",  
                "direccion_envio": "$direccion_envio",
                "precio_compra": "$precio_compra",
                "productos": "$productos",
                "fecha_compra": "$fecha_compra"
            }
        },
        {
            "$out": "Compras_dia_determinado"  
        }
    ]
    
    Compra.aggregate(pipeline)  


guardar_compras_por_fecha("28-08-2024")
